# Markowitz Portfolio's Theory #

### Building the Efficient Frontier ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Optimization
from scipy.optimize import minimize

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from config import get_tickers
from data_downloader import get_market_data

In [2]:
tickers = get_tickers(mod="2.2")

tickers

In [3]:
# Import data
df_returns = pd.DataFrame()

for ticker in tickers:
    df = get_market_data(
        ticker=ticker, 
        start_date='2015-01-01', 
        end_date='2025-01-01', 
        returns=True
    )
    
    returns = df['returns'].rename(ticker)
    
    df_returns = pd.concat([df_returns, returns], axis=1)
    
    print(f'Data Ready for {ticker}')


In [4]:
df_returns

In [5]:
# Theoretically, we could use the average as the expected returns (these are daily returns)
mean_returns = df_returns.mean()
mean_returns.name = 'mean_returns'

mean_returns

In [6]:
# The volatility is calculated with the standard deviations (also daily volatility)
volatility = df_returns.dropna().std()
volatility.name = 'volatility'

volatility

In [7]:
# Covariance Matrix
covariances = df_returns.dropna().cov()

covariances

In [8]:
# Correlation Matrix
correlations = df_returns.corr()

correlations

In [9]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns.cumsum(), label=df_returns.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('$r_t$')
plt.legend()

# Show
plt.grid(True)
plt.show()

The Markowitz Theory establishes that a portfolio's returns and variance are defined by the following equations:

Portfolio Returns: $ \mu_P = \sum_{i=1}^{n}{\omega_i\mu_i} $

Portfolio Variance: $ \sigma_P^2 = \sum_{i=1}^{n}\sum_{j=1}^{n}\omega_i\omega_j\gamma_{ij}$

In [10]:
# To create random portfolios, first we need to create random weights:
def rand_weights(n):
    """Produces n random weights that sum to 1 """
    k = np.random.rand(n)
    return k / sum(k)

In [11]:
# An example
rand_weights(5)

In [12]:
### This function create a random portfolio based on random weights
def random_portfolio(
        expected_returns, 
        covariance_matrix,
):
    # Generate Random Weights
    weights = rand_weights(len(expected_returns))
    
    # Calculate the Portfolio's Returns
    portfolio_returns = np.dot(weights, expected_returns)
    
    # Calculate the Portfolio's Risk
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    portfolio_std = np.sqrt(portfolio_variance)
    
    return portfolio_returns, portfolio_std

In [13]:
# Calculate a portfolio
returns_i, risk_i = random_portfolio(mean_returns, covariances)

print(f"The Random Portfolio's Return is: {returns_i.round(3)}")
print(f"The Random Portfolio's Volatility is: {risk_i.round(3)}")

In [14]:
# We can use this function to generate several random portfolios
def generate_random_portfolios(
        n_portfolios, 
        expected_returns, 
        cov_matrix
):

    # Lists to store the portfolios' information
    means = []
    stds = []

    # Generate the portfolios
    for _ in range(n_portfolios):
        mean, std = random_portfolio(expected_returns.values.flatten(), cov_matrix)
        means.append(mean)
        stds.append(std)
    
    # Store them in a DataFrame
    portfolios = pd.DataFrame({
        'Mean Return': means,
        'Std Dev': stds
    })
    
    return portfolios

In [15]:
# Generate Random Portfolios
q1 = 1000
r_portfolios = generate_random_portfolios(q1, mean_returns, covariances)

r_portfolios

In [16]:
# Portfolios Plot
plt.figure(figsize=(10, 6))
plt.scatter(r_portfolios['Std Dev'], r_portfolios['Mean Return'], color='gray', alpha=0.8, label='Portfolios')

# Config
plt.title('Portfolios')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [17]:
# We can create the Equal-Weighted Portfolio to compare it with the other portfolios
def equal_weighted_portfolio(
        expected_returns, 
        covariance_matrix
):
    # Generate the Equal Weights
    n = len(expected_returns)
    weights = np.ones(n) / n  

    # Calculate the Portfolio's Returns
    portfolio_return = np.dot(weights, expected_returns)
    
    # Calculate the Portfolio's Risk
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    
    return portfolio_return, portfolio_volatility

In [18]:
# Calculate the EWP
returns_e, risk_e = equal_weighted_portfolio(mean_returns, covariances)

print(f"The Equal-Weighted Portfolio's Return is: {returns_e.round(3)}")
print(f"The Equal-Weighted Portfolio's Volatility is: {risk_e.round(3)}")

In [19]:
# Portfolios Plot
plt.figure(figsize=(10, 6))
plt.scatter(r_portfolios['Std Dev'], r_portfolios['Mean Return'], color='gray', alpha=0.8, label='Portfolios')
plt.scatter(risk_e, returns_e, color='red', s=50, label='Equal-Weighted Portfolio')  


# Config
plt.title('Portfolios')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.legend()

# Show
plt.grid(True)
plt.show()

### Efficient Frontier ###

In [20]:
# Optimization Functions
def portfolio_performance(
        weights, 
        expected_returns, 
        covariance_matrix
):
    # Portfolio's Returns
    portfolio_return = np.dot(weights, expected_returns)

    #Portfolio's Volatility
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    
    return portfolio_return, portfolio_volatility
    
# Minimizing Portfolio
def minimize_volatility(
        weights, 
        expected_returns, 
        covariance_matrix
):
    return portfolio_performance(weights, expected_returns, covariance_matrix)[1]

# Function that generates the portfolios that are located in the Efficient Frontier
def get_efficient_frontier(
        expected_returns, 
        covariance_matrix, 
        num_portfolios=100
):
    results = np.zeros((2, num_portfolios))
    target_returns = np.linspace(expected_returns.min(), expected_returns.max(), num_portfolios)
    
    for i, target in enumerate(target_returns):
        constraints = (
                {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},                         # weights must sum one
            {'type': 'eq', 'fun': lambda x: np.dot(x, expected_returns) - target}       # portfolio returns
        )
        
        bounds = tuple((-1, 1) for _ in range(len(expected_returns)))                   # no short if bounds [0,1]
        
        # Initial Portfolio
        initial_guess = len(expected_returns) * [1. / len(expected_returns)]
        
        opt = minimize(
            minimize_volatility, 
            initial_guess, 
            args=(expected_returns, covariance_matrix),
            method='SLSQP', 
            bounds=bounds, 
            constraints=constraints
        )
        
        if opt.success:
            results[0, i] = target
            results[1, i] = opt.fun

    return results

In [21]:
# Calculate the EF
efficient_frontier = get_efficient_frontier(mean_returns, covariances)

In [22]:
# Portfolios Plot
plt.figure(figsize=(10, 6))
plt.scatter(r_portfolios['Std Dev'], r_portfolios['Mean Return'], color='gray', alpha=0.7, label='Random Portfolios')
plt.scatter(risk_e, returns_e, color='red', s=50, label='Equal-Weighted Portfolio')  
plt.plot(efficient_frontier[1, :], efficient_frontier[0, :], label='Efficient Frontier', color='black')


# Config
plt.title('Efficient Frontier')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.legend()

# Show
plt.grid(True)
plt.show()

The equation of the Efficient Frontier will have the next form:

1) $ \sigma_P^2 = \pi_0 - \pi_1\mu_P + \pi_2\mu_P^2 $

The coefficients of the equation will have the next form:

1) $ \pi_0 = \frac{A}{D} $
2) $ \pi_1 = \frac{2B}{D} $
3) $ \pi_2 = \frac{C}{D} $

The next equations define the components of the coefficients:

1) $ A = \mu^⊤\Sigma^{-1}\mu $
2) $ B = \mu^⊤\Sigma^{-1}\iota $
3) $ C = \iota^⊤\Sigma^{-1}\iota" $
4) $ D = AC-B^2 $


In [23]:
# So let us get the components
q2 = len(mean_returns)                                      # Number of Stocks
mu = mean_returns.values.flatten().reshape(-1, 1)           # Expected Returns
Sigma = covariances.values                                  # Covariance Matrix
Sigma_inv = np.linalg.inv(Sigma)                            # Inverse Covariance Matrix
iota = np.ones((q2, 1))                                     # Vector of Ones

In [24]:
# And now get the coefficients of the Efficient Frontier

A = np.dot(np.dot(mu.T, Sigma_inv), mu)
B = np.dot(np.dot(iota.T, Sigma_inv), mu)
C = np.dot(np.dot(iota.T, Sigma_inv), iota)
D = (A*C - B*B)

print(f"This is A: {A[0][0]}")
print(f"This is B: {B[0][0]}")
print(f"This is C: {C[0][0]}")
print(f"This is D: {D[0][0]}")

In [25]:
# Then the equation
pi_0 = A/D
pi_1 = 2*B/D
pi_2 = C/D

print(f"This is the first coefficient: {pi_0[0][0]}")
print(f"This is the second coefficient: {pi_1[0][0]}")
print(f"This is the third coefficient: {pi_2[0][0]}")

In [26]:
# Now let us get the values of the efficient frontier
def eff_equation(mu_P):
    return np.sqrt((pi_0 - pi_1 * mu_P + pi_2 * mu_P**2))

# Create a rango of values for mu_P
mu_P_values = np.linspace(0.00025, 0.001, 400)

# Evaluate the equation for mu_P values
sigma_P_values = eff_equation(mu_P_values).reshape(-1, 1)

In [27]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.scatter(r_portfolios['Std Dev'], r_portfolios['Mean Return'], color='grey', alpha=0.7, label='Portfolios')
plt.scatter(risk_e, returns_e, color='red', s=50, label='Equal-Weighted Portfolio') 
plt.plot(sigma_P_values, mu_P_values, color='black')


# Config
plt.title('Efficient Frontier and Portfolios')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.legend()

# Show
plt.grid(True)
plt.show()


In [28]:
# Let us check if this equation is indeed correct; for the equal-weighted portfolio's volatility
risk_e_optimal = eff_equation(returns_e)[0][0]

print(f"The Equal-Weighted Portfolio's Returns: {returns_e}")
print(f"The Equal-Weighted Portfolio's Volatility: {risk_e}")
print(f"The Optimal Volatility: {risk_e_optimal}")

In [29]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.scatter(r_portfolios['Std Dev'], r_portfolios['Mean Return'], color='grey', alpha=0.7, label='Portfolios')
plt.plot(sigma_P_values, mu_P_values, color='black')
plt.scatter(risk_e, returns_e, color='red', s=50, label='Equal-Weighted Portfolio')  
plt.scatter(risk_e_optimal, returns_e, color='yellow', s=50, label='Optimal Portfolio') 

# Config
plt.title('Efficient Frontier and Portfolios')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.legend()

# Show
plt.grid(True)
plt.show()


To find the minimum variance portfolio, we can use the following equation:

Returns: $ \mu_{P_{min}} = \frac{\pi_1}{2\pi_2} $

Variance: $ \sigma_{P_{min}}^2 = \pi_0 - \frac{\pi_1^2}{4\pi_2} $ 

In [30]:
# Get the MVP
min_returns = pi_1/(2*pi_2)
min_volatility = np.sqrt(pi_0 - ((pi_1**2)/(4*pi_2)))

print(f"The MVP Returns are: {min_returns[0][0]}")
print(f"The MVP Volatility is: {min_volatility[0][0]}")

In [31]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.scatter(r_portfolios['Std Dev'], r_portfolios['Mean Return'], color='grey', alpha=0.7, label='Portfolios')
plt.plot(sigma_P_values, mu_P_values, color='black')
plt.scatter(risk_e, returns_e, color='red', s=50, label='Equal-Weighted Portfolio')  
plt.scatter(risk_e_optimal, returns_e, color='orange', s=50, label='Equal-Weighted Optimal Portfolio') 
plt.scatter(min_volatility, min_returns, color='purple', s=50, label='MVP Portfolio') 

# Config
plt.title('Efficient Frontier and Portfolios')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.legend()

# Show
plt.grid(True)
plt.show()